In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, chi2_contingency

In [5]:
df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')  # Adjust path if needed
df.head()

C:\Users\saron\AppData\Local\Temp\ipykernel_14040\3789772269.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')  # Adjust path if needed


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [6]:
df['HasClaim'] = df['TotalClaims'] > 0

# Claim Frequency (overall)
claim_frequency = df['HasClaim'].mean()

# Claim Severity (only where claims occurred)
claim_severity = df[df['HasClaim']]['TotalClaims'].mean()

# Margin
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

print("Claim Frequency:", round(claim_frequency, 4))
print("Claim Severity:", round(claim_severity, 2))

Claim Frequency: 0.0028
Claim Severity: 23273.39


## Step 3: Data Segmentation for A/B Testing


In [9]:
print(df['Province'].value_counts())
print(df['PostalCode'].value_counts().head(10))
print(df['Gender'].value_counts())

Province
Gauteng          393865
Western Cape     170796
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52718
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64
PostalCode
2000    133498
122      49171
7784     28585
299      25546
7405     18518
458      13775
8000     11794
2196     11048
470      10226
7100     10161
Name: count, dtype: int64
Gender
Not specified    940990
Male              42817
Female             6755
Name: count, dtype: int64


In [10]:
# Pick top 2 postal codes by count for testing
top_postal_codes = df['PostalCode'].value_counts().nlargest(2).index.tolist()

# Filter dataset to only include those top postal codes
df_zip_ab = df[df['PostalCode'].isin(top_postal_codes)].copy()
df_zip_ab['ZipGroup'] = df_zip_ab['PostalCode'].apply(lambda x: 'A' if x == top_postal_codes[0] else 'B')

print(df_zip_ab['ZipGroup'].value_counts())


ZipGroup
A    133498
B     49171
Name: count, dtype: int64


In [11]:
df_gender_ab = df[df['Gender'].isin(['Male', 'Female'])].copy()
print(df_gender_ab['Gender'].value_counts())

Gender
Male      42817
Female     6755
Name: count, dtype: int64


In [12]:
import scipy.stats as stats

# Province-wise groups for claim frequency
province_groups = [group['HasClaim'].values for _, group in df.groupby('Province')]

# One-way ANOVA
f_freq, p_freq = stats.f_oneway(*province_groups)
print(f"ANOVA (Claim Frequency across Provinces): F = {f_freq:.4f}, p = {p_freq:.4f}")

ANOVA (Claim Frequency across Provinces): F = 13.0251, p = 0.0000


In [13]:
# Filter only rows with a claim
df_with_claims = df[df['HasClaim'] == 1]

# Province-wise claim severity
severity_groups = [group['TotalClaims'].values for _, group in df_with_claims.groupby('Province')]

f_sev, p_sev = stats.f_oneway(*severity_groups)
print(f"ANOVA (Claim Severity across Provinces): F = {f_sev:.4f}, p = {p_sev:.4f}")


ANOVA (Claim Severity across Provinces): F = 4.8302, p = 0.0000


In [14]:
# Province-wise margin
margin_groups = [group['Margin'].values for _, group in df.groupby('Province')]

f_margin, p_margin = stats.f_oneway(*margin_groups)
print(f"ANOVA (Margin across Provinces): F = {f_margin:.4f}, p = {p_margin:.4f}")


ANOVA (Margin across Provinces): F = 3.2226, p = 0.0011


In [15]:
from scipy.stats import ttest_ind

# T-test on Claim Frequency (binary outcome, use mean difference)
group_a_freq = df_zip_ab[df_zip_ab['ZipGroup'] == 'A']['HasClaim']
group_b_freq = df_zip_ab[df_zip_ab['ZipGroup'] == 'B']['HasClaim']
t_freq_zip, p_freq_zip = ttest_ind(group_a_freq, group_b_freq)
print(f"T-test (Claim Frequency by ZipGroup): t = {t_freq_zip:.4f}, p = {p_freq_zip:.4f}")

# T-test on Claim Severity (only where claim occurred)
df_zip_claims = df_zip_ab[df_zip_ab['HasClaim'] == 1]
group_a_sev = df_zip_claims[df_zip_claims['ZipGroup'] == 'A']['TotalClaims']
group_b_sev = df_zip_claims[df_zip_claims['ZipGroup'] == 'B']['TotalClaims']
t_sev_zip, p_sev_zip = ttest_ind(group_a_sev, group_b_sev)
print(f"T-test (Claim Severity by ZipGroup): t = {t_sev_zip:.4f}, p = {p_sev_zip:.4f}")

# T-test on Margin
group_a_margin = df_zip_ab[df_zip_ab['ZipGroup'] == 'A']['Margin']
group_b_margin = df_zip_ab[df_zip_ab['ZipGroup'] == 'B']['Margin']
t_margin_zip, p_margin_zip = ttest_ind(group_a_margin, group_b_margin)
print(f"T-test (Margin by ZipGroup): t = {t_margin_zip:.4f}, p = {p_margin_zip:.4f}")


T-test (Claim Frequency by ZipGroup): t = inf, p = nan
T-test (Claim Severity by ZipGroup): t = 0.4214, p = 0.6736
T-test (Margin by ZipGroup): t = 1.2933, p = 0.1959


In [16]:
print(df_zip_ab.groupby('ZipGroup')['HasClaim'].mean())


ZipGroup
A    0.003641
B    0.004271
Name: HasClaim, dtype: float64


In [17]:
# Frequency
group_male_freq = df_gender_ab[df_gender_ab['Gender'] == 'Male']['HasClaim']
group_female_freq = df_gender_ab[df_gender_ab['Gender'] == 'Female']['HasClaim']
t_freq_gender, p_freq_gender = ttest_ind(group_male_freq, group_female_freq)
print(f"T-test (Claim Frequency by Gender): t = {t_freq_gender:.4f}, p = {p_freq_gender:.4f}")

# Severity
df_gender_claims = df_gender_ab[df_gender_ab['HasClaim'] == 1]
group_male_sev = df_gender_claims[df_gender_claims['Gender'] == 'Male']['TotalClaims']
group_female_sev = df_gender_claims[df_gender_claims['Gender'] == 'Female']['TotalClaims']
t_sev_gender, p_sev_gender = ttest_ind(group_male_sev, group_female_sev)
print(f"T-test (Claim Severity by Gender): t = {t_sev_gender:.4f}, p = {p_sev_gender:.4f}")

# Margin
group_male_margin = df_gender_ab[df_gender_ab['Gender'] == 'Male']['Margin']
group_female_margin = df_gender_ab[df_gender_ab['Gender'] == 'Female']['Margin']
t_margin_gender, p_margin_gender = ttest_ind(group_male_margin, group_female_margin)
print(f"T-test (Margin by Gender): t = {t_margin_gender:.4f}, p = {p_margin_gender:.4f}")


T-test (Claim Frequency by Gender): t = -inf, p = nan
T-test (Claim Severity by Gender): t = -0.4191, p = 0.6760
T-test (Margin by Gender): t = -0.2104, p = 0.8334


In [18]:
print(df_gender_ab.groupby('Gender')['HasClaim'].mean())


Gender
Female    0.002073
Male      0.002195
Name: HasClaim, dtype: float64
